# Potential Talents
---
## Summary
**Background:**

As a talent sourcing and management company, we are interested in finding talented individuals for sourcing these candidates to technology companies. Finding talented candidates is not easy, for several reasons. The first reason is one needs to understand what the role is very well to fill in that spot, this requires understanding the client’s needs and what they are looking for in a potential candidate. The second reason is one needs to understand what makes a candidate shine for the role we are in search for. Third, where to find talented individuals is another challenge.

The nature of our job requires a lot of human labor and is full of manual operations. Towards automating this process we want to build a better approach that could save us time and finally help us spot potential candidates that could fit the roles we are in search for. Moreover, going beyond that for a specific role we want to fill in we are interested in developing a machine learning powered pipeline that could spot talented individuals, and rank them based on their fitness.

We are right now semi-automatically sourcing a few candidates, therefore the sourcing part is not a concern at this time but we expect to first determine best matching candidates based on how fit these candidates are for a given role. We generally make these searches based on some keywords such as “full-stack software engineer”, “engineering manager” or “aspiring human resources” based on the role we are trying to fill in. These keywords might change, and you can expect that specific keywords will be provided to you.

Assuming that we were able to list and rank fitting candidates, we then employ a review procedure, as each candidate needs to be reviewed and then determined how good a fit they are through manual inspection. This procedure is done manually and at the end of this manual review, we might choose not the first fitting candidate in the list but maybe the 7th candidate in the list. If that happens, we are interested in being able to re-rank the previous list based on this information. This supervisory signal is going to be supplied by starring the 7th candidate in the list. Starring one candidate actually sets this candidate as an ideal candidate for the given role. Then, we expect the list to be re-ranked each time a candidate is starred.

**Data Description**

The data comes from our sourcing efforts. We removed any field that could directly reveal personal details and gave a unique identifier for each candidate.

Attributes:
- id : unique identifier for candidate (numeric)
- job_title : job title for candidate (text)
- location : geographical location for candidate (text)
- connections: number of connections candidate has, 500+ means over 500 (text)

Output (desired target):
- fit - how fit the candidate is for the role? (numeric, probability between 0-1)

Keywords: “Aspiring human resources” or “seeking human resources”

**Goal(s):**

- Predict how fit the candidate is based on their available information (variable fit)

**Success Metric(s):**

- Rank candidates based on a fitness score.
- Re-rank candidates when a candidate is starred.

**Bonus(es):**

- We are interested in a robust algorithm, tell us how your solution works and show us how your ranking gets better with each starring action.
- How can we filter out candidates which in the first place should not be in this list?
- Can we determine a cut-off point that would work for other roles without losing high potential candidates?
- Do you have any ideas that we should explore so that we can even automate this procedure to prevent human bias?

# Exploratory Data Analysis

## Import Libraries

In [1]:
# Standards
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from scipy import stats

# for doc2vec
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

# for cleaning
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
# If running for the first time, download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.preprocessing import MinMaxScaler

# for BERT
from transformers import BertTokenizer, BertModel
import torch

# Warnings
import warnings

# Suppress deprecation warnings
#warnings.filterwarnings('ignore', category=FutureWarning)
#warnings.filterwarnings('ignore', category=UserWarning)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
c:\Users\danie\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import Data

In [2]:
# Load the data from a CSV file
dataset = pd.read_csv('potential-talents.csv')

# Extract only 2 feature columns
x = dataset.iloc[:, [0,1,3]].values
y = dataset.iloc[:, 4].values

# Create a DataFrame with the feature columns
df = pd.DataFrame(x, columns=['id','job_title', 'connection'])

print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

x shape: (104, 3)
y shape: (104,)


# Data Preprocessing

## Missing Data

In [3]:
# Handle missing data
missing_data = dataset.isnull().sum()
print("Missing Data:")
print(missing_data)

# Impute missing values
#imputer = SimpleImputer(strategy='mean')  # Use mean imputation for numeric features
#dataset[numeric_features] = imputer.fit_transform(dataset[numeric_features])

Missing Data:
id              0
job_title       0
location        0
connection      0
fit           104
dtype: int64


## Cleaning



### job_title

In [4]:
# Initialize lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Define preprocesser
def preprocess(document):

    document = document.lower() # Convert text to lower case
    document = re.sub(r'\d+', '', document) # Remove numbers
    document = re.sub(r'\W', ' ', document) # Remove punctuation
    document = document.strip() # Remove leading/trailing white space

    tokens = nltk.word_tokenize(document) # Tokenize into words
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords

    # Lemmatize or stem the words
    # For lemmatization:
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # For stemming:
    # tokens = [stemmer.stem(word) for word in tokens]

    document = " ".join(tokens) # Join the processed words back into a single string

    return document

In [5]:
# Clean columns
df['job_title'] = df['job_title'].apply(preprocess)

# Preprocess the words before getting their vectors
w1 = preprocess("Aspiring human resources")
w2 = preprocess("seeking human resources")

print(f"w1 becomes: {w1}")
print(f"w2 becomes: {w2}")

w1 becomes: aspiring human resource
w2 becomes: seeking human resource


### connection

In [6]:
# Make the "connection" column as a numeric column by making "500+" to "500" and leaving the rest
df['connection'] = df['connection'].replace('500+ ', '500').astype(int)
df['connection'] = df['connection'].replace('500+', '500').astype(int)


In [7]:
# Scale the "connection" column

# Initialize the scaler
scaler = MinMaxScaler()

# Scale the 'connections' column
df['connection_scaled'] = scaler.fit_transform(df[['connection']])

# Natural Language Processing

## Doc2Vec

In [8]:
# Create doc2vec embeddings for all job titles, and w1 (chosen phrases)

# Tokenize the job_titles
data = [word_tokenize(title) for title in df['job_title']]

# Create tagged documents
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(data)]

# Set-up the model
model = Doc2Vec(vector_size=20,      # Dimensionality of the feature vectors
                window=2,            # The maximum distance between the current and predicted word within a sentence.
                min_count=1,         # Ignores all words with total frequency lower than this.
                workers=4,           # Use these many worker threads to train the model
                epochs=100)          # Number of iterations (epochs) over the corpus

# Build vocabulary from a sequence of sentences
model.build_vocab(tagged_data)

# Train the model
model.train(tagged_data,
            total_examples=model.corpus_count,
            epochs=model.epochs)

# Extract vectors of all job_titles
vectors = [model.dv[i] for i in range(len(df['job_title']))]

# now 'vectors' contains the doc2vec vectors for each job_title

# Infer the vector for each chosen phrase
v1 = model.infer_vector(word_tokenize(w1))
v2 = model.infer_vector(word_tokenize(w2))

In [9]:
# Calculate Cosine Similarity between w1 and each job_title

# Initialize an empty list to store the similarities
similarities = []

# Loop through each job title vector
for vec in vectors:
    # Reshape vectors to 2D arrays necessary for cosine_similarity function
    vec = np.array(vec).reshape(1, -1)
    v1 = np.array(v1).reshape(1, -1)

    # Calculate similarity and append to the list
    sim = cosine_similarity(vec, v1)
    similarities.append(sim[0][0])

# Now 'similarities' is a list of cosine similarities between 'w1' and each job title.

# Add the 'similarities' list as a new column to the dataframe
df['similarity'] = similarities

print(f"Size of Similarities: {len(similarities)}")

Size of Similarities: 104


## BERT
(Bidirectional Encoder Representations from Transformers)

In [10]:
# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')

# Function to generate BERT embeddings
def generate_bert_embeddings(text):
    # Add the special tokens for BERT
    marked_text = "[CLS] " + text + " [SEP]"

    # Tokenize our sentence with the BERT tokenizer.
    tokenized_text = tokenizer.tokenize(marked_text)

    # Map the token strings to their vocabulary indices.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # Mark each token as belonging to sentence "1".
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()

    # Predict hidden states features for each layer
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # `outputs` is a tuple, we only need the token embeddings from the first element of the tuple
        encoded_layers = outputs[0]

    # Get the embeddings of the [CLS] token as the text embedding
    text_embedding = encoded_layers[0][0]

    return text_embedding.numpy()  # Convert tensor to numpy array for easier use

# Generate BERT embeddings for all job titles and store in a list
bert_embeddings = df['job_title'].apply(generate_bert_embeddings).tolist()

# Generate BERT embedding for each chosen phrase
w1_bert = generate_bert_embeddings(w1)
w2_bert = generate_bert_embeddings(w2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# Initialize an empty list to store the similarities
bert_similarities = []

# Loop through each BERT job title vector
for bert_vec in bert_embeddings:
    # Reshape vectors to 2D arrays necessary for cosine_similarity function
    bert_vec = np.array(bert_vec).reshape(1, -1)
    w1_bert = np.array(w1_bert).reshape(1, -1)

    # Calculate similarity and append to the list
    sim = cosine_similarity(bert_vec, w1_bert)
    bert_similarities.append(sim[0][0])

# Now 'bert_similarities' is a list of cosine similarities between 'w1' and each job title

# Add the 'similarities' list as a new column to the dataframe
df['bert_similarity'] = bert_similarities

print(f"Size of bert_Similarities: {len(bert_similarities)}")

Size of bert_Similarities: 104


## Weighted Sum

In [12]:
# Set weights here
weight_similarity = 0.8
weight_connection = 0.2

# Ensure weights add up to 1
assert weight_similarity + weight_connection == 1, "Weights should add up to 1"

# Create the weighted sum column
df['ranking'] = weight_similarity * df['similarity'] + weight_connection * df['connection_scaled']


In [13]:
# Set BERT weights here
weight_similarity_bert = 0.8
weight_connection = 0.2

# Ensure weights add up to 1
assert weight_similarity_bert + weight_connection == 1, "Weights should add up to 1"

# Create the weighted sum column
df['bert_ranking'] = weight_similarity_bert * df['bert_similarity'] + weight_connection * df['connection_scaled']

# Evaluation

In [14]:
# Sorting by Doc2Vec ranking
doc2vec_ranking = df.sort_values('ranking', ascending=False)

# Sorting by BERT ranking
bert_ranking = df.sort_values('bert_ranking', ascending=False)

# Select specific columns
doc2vec_ranking = doc2vec_ranking[['id', 'job_title', 'ranking']]
bert_ranking = bert_ranking[['id', 'job_title', 'bert_ranking']]

# Print top 10 
print("Top 10 rankings based on Doc2Vec:\n")
print(doc2vec_ranking.head(10).to_string(index=False))
print("\nTop 10 rankings based on BERT:\n")
print(bert_ranking.head(10).to_string(index=False))


Top 10 rankings based on Doc2Vec:

 id                                                                            job_title  ranking
  8                                                                 hr senior specialist 0.983765
 67                                      human resource staffing recruiting professional 0.983357
 85                        rrp brand portfolio executive jti japan tobacco international 0.981902
 75           nortia staffing seeking human resource payroll administrative professional 0.981445
 71                                            human resource generalist scottmadden inc 0.981404
 69                                director human resource north america groupe beneteau 0.981355
104                                           director administration excellence logging 0.980215
103                                                                   always set success 0.979335
 64 svp chro marketing communication csr officer engie houston woodland energy gphr